# NoteBook 1
## Lab Instructions here :

0. Ensure Azure KeyVault is created.
    -[Create Azure KeyVault](https://learn.microsoft.com/en-us/azure/key-vault/general/quick-create-portal)
1. Install Azure Synapse & Create a small serverless Apache Spark Pool
    -[Create Synapse Workspace](https://learn.microsoft.com/en-us/azure/synapse-analytics/quickstart-create-workspace)
    -[Create Serverless Apache Spark Pool](https://learn.microsoft.com/en-us/azure/synapse-analytics/quickstart-create-apache-spark-pool-portal)
    -[Configure Access to Azure KeyVault](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities?pivots=programming-language-python#configure-access-to-azure-key-vault)
    -[Another Article for Configuring KeyVault Access](https://learn.microsoft.com/en-us/azure/synapse-analytics/machine-learning/tutorial-configure-cognitive-services-synapse#create-a-key-vault-and-configure-secrets-and-access)
    - [Documentation](https://learn.microsoft.com/en-us/azure/cognitive-services/big-data/getting-started#azure-synapse-analytics-optional)
2. Configure Cognitive Services
3. Edit the parameters in the notebook for the following
    - Azure Search
    - Azure SQL 
    - Azure Cognitive Services Key
4. Make sure the storage account is hydrated with PDF (create a directory called notebook1_pdf)
5. Run Cell to Process a document
6. Run Cell to Observe the output
7. Run Cell to write to Azure Search Index
    - Understand the schema and index being written to 
8. Run Cell to write to Azure SQL Database

## Appendix
1. [SynapseML Documentation](https://microsoft.github.io/SynapseML/docs/features/cognitive_services/CognitiveServices%20-%20Overview/)
2. [Microsoft Spark Utilities](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities)
3. [Configure Access to Azure Key Vault](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/microsoft-spark-utilities?pivots=programming-language-python#configure-access-to-azure-key-vault)
4. https://learn.microsoft.com/en-us/azure/synapse-analytics/cognitive-services-with-synapseml-overview

In [ ]:
import synapse.ml
from synapse.ml.cognitive import *
from notebookutils import mssparkutils

# A general Cognitive Services key for Text Analytics and Computer Vision (or use separate keys that belong to each service)
cognitive_service_key = mssparkutils.credentials.getSecret("ADD_YOUR_KEY_VAULT_NAME", "ADD_YOUR_SERVICE_KEY","ADD_YOUR_KEY_VAULT_LINKED_SERVICE_NAME")
# A Bing Search v7 subscription key
bingsearch_service_key = mssparkutils.credentials.getSecret("ADD_YOUR_KEY_VAULT_NAME", "ADD_YOUR_BING_SEARCH_KEY","ADD_YOUR_KEY_VAULT_LINKED_SERVICE_NAME")
# An Anomaly Dectector subscription key
anomalydetector_key = mssparkutils.credentials.getSecret("ADD_YOUR_KEY_VAULT_NAME", "ADD_YOUR_ANOMALY_KEY","ADD_YOUR_KEY_VAULT_LINKED_SERVICE_NAME")


# Run the sample below to 

In [ ]:
from mmlspark.cognitive import *
from pyspark.sql.functions import col

# Add your region and subscription key from the Language service (or a general Cognitive Service key)
# If using a multi-region Cognitive Services resource, delete the placeholder text: service_region = ""
service_key = "ADD-SUBSCRIPTION-KEY-HERE"
service_region = "ADD-SERVICE-REGION-HERE"

df = spark.createDataFrame([
  ("I am so happy today, its sunny!", "en-US"),
  ("I am frustrated by this rush hour traffic", "en-US"),
  ("The cognitive services on spark aint bad", "en-US"),
], ["text", "language"])

sentiment = (TextSentiment()
    .setTextCol("text")
    .setLocation(service_region)
    .setSubscriptionKey(service_key)
    .setOutputCol("sentiment")
    .setErrorCol("error")
    .setLanguageCol("language"))

results = sentiment.transform(df)

# Show the results in a table
display(results.select("text", col("sentiment")[0].getItem("score").alias("sentiment")))